In [1]:
import os
import urllib.request
from tqdm import tqdm
import lzma


# --- Config ---
DATA_DIR = "pile_of_law_data"
os.makedirs(DATA_DIR, exist_ok=True)

files_to_download = [
    "train.atticus_contracts.0.jsonl.xz",     # Real-world contracts
    "train.resource_contracts.jsonl.xz",      # Public resource agreements 
    "train.tos.jsonl.xz",                     # Corporate-style ToS 
    "train.cfpb_cc.jsonl.xz",                 # Consumer protection/credit agreements
    "train.uscode.jsonl.xz",                  # Federal statutes 
    "train.state_code.jsonl.xz",              # State-level laws 
    "train.cfr.jsonl.xz",                     # Federal regulations 
]

BASE_URL = "https://huggingface.co/datasets/pile-of-law/pile-of-law/resolve/main/data/"

for idx, filename in enumerate(files_to_download):
    url = BASE_URL + filename
    local_path = os.path.join(DATA_DIR, filename)

    with tqdm(total=1, position=0, desc=f"Downloading {filename}", leave=True, bar_format="{desc}") as progress:
        if os.path.exists(local_path):
            progress.set_description(f"Already exists: {filename}")
            continue

        try:
            urllib.request.urlretrieve(url, local_path)
            progress.set_description(f"Saved: {filename}")
        except Exception as e:
            progress.set_description(f"Error: {filename} ({e})")
        
        try:
            with lzma.open(local_path) as test_file:
                test_file.read(1)  
        except Exception:
            progress.set_description(f"Corrupt file: {filename}")
            os.remove(local_path)
            continue

Already exists: train.atticus_contracts.0.jsonl.xz: 
Already exists: train.resource_contracts.jsonl.xz: 
Already exists: train.tos.jsonl.xz: 
Already exists: train.cfpb_cc.jsonl.xz: 
Already exists: train.uscode.jsonl.xz: 
Already exists: train.state_code.jsonl.xz: 
Already exists: train.cfr.jsonl.xz: 


In [2]:
import os
import lzma
from tqdm import tqdm

DATA_DIR = "pile_of_law_data"
OUTPUT_DIR = "pile_of_law_decompressed"  

os.makedirs(OUTPUT_DIR, exist_ok=True)

for filename in os.listdir(DATA_DIR):
    if filename.endswith(".jsonl.xz"):
        input_path = os.path.join(DATA_DIR, filename)
        output_path = os.path.join(OUTPUT_DIR, filename.replace(".xz", ""))
        
        if os.path.exists(output_path):
            tqdm.write(f"Skipping {filename} - already decompressed")
            continue

        tqdm.write(f"Decompressing {filename} → {os.path.basename(output_path)}")

        try:
            with lzma.open(input_path, "rb") as compressed, open(output_path, "wb") as out_file:
                out_file.write(compressed.read())

            tqdm.write(f"Saved: {output_path}")

        except Exception as e:
            tqdm.write(f"Failed to decompress {filename}: {e}")

print(f"Decompressed files saved to '{OUTPUT_DIR}' directory.")

Skipping train.resource_contracts.jsonl.xz - already decompressed
Skipping train.state_code.jsonl.xz - already decompressed
Skipping train.atticus_contracts.0.jsonl.xz - already decompressed
Skipping train.cfpb_cc.jsonl.xz - already decompressed
Skipping train.tos.jsonl.xz - already decompressed
Skipping train.cfr.jsonl.xz - already decompressed
Skipping train.uscode.jsonl.xz - already decompressed
Decompressed files saved to 'pile_of_law_decompressed' directory.


In [3]:
import json
import os
import re
from pathlib import Path

OUTPUT_DIR = "pile_of_law_decompressed"
PARSED_DIR = "parsed_docs_batches"
os.makedirs(PARSED_DIR, exist_ok=True)

# Map raw filenames to clean source labels
SOURCE_NAME_MAP = {
    "train.atticus_contracts.0": "Atticus Contracts",
    "train.resource_contracts": "Resource Contracts",
    "train.tos": "Terms of Service",
    "train.cfpb_cc": "Consumer Credit Agreements",
    "train.uscode": "US Code",
    "train.state_code": "State Codes",
    "train.cfr": "Code of Federal Regulations"
}

def clean_chunk_text(text):
    if not isinstance(text, str):
        return ""
    text = text.replace('\x00', '')
    text = re.sub(r"[^\x09\x0A\x0D\x20-\uD7FF\uE000-\uFFFD]", "", text)
    text = re.sub(r'-{5,}', '', text)             # remove visual dividers
    text = re.sub(r'\n\s*\n+', '\n\n', text)      # collapse extra blank lines
    text = re.sub(r'[ \t]+', ' ', text)           # normalize intra-line whitespace
    text = text.replace('\u00A0', ' ')            # non-breaking spaces
    return text.strip()

batch_size = 10000
doc_buffer = []
file_count = 0
doc_count = 0

for file in sorted(os.listdir(OUTPUT_DIR)):
    if file.endswith(".jsonl"):
        filepath = os.path.join(OUTPUT_DIR, file)
        raw_source = Path(file).stem
        source = SOURCE_NAME_MAP.get(raw_source, raw_source)

        with open(filepath, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    doc = json.loads(line)

                    if "text" not in doc:
                        print(f"Warning: No 'text' field in document from {file}")
                    elif not isinstance(doc["text"], str):
                        print(f"Warning: 'text' field is not a string in document from {file}")

                    doc_buffer.append({
                        "source": source,
                        "text": clean_chunk_text(doc.get("text", ""))
                    })
                    doc_count += 1

                    if len(doc_buffer) >= batch_size:
                        file_count += 1
                        batch_path = os.path.join(PARSED_DIR, f"parsed_docs_{file_count:04d}.json")
                        with open(batch_path, 'w', encoding='utf-8') as out_f:
                            json.dump(doc_buffer, out_f)
                        print(f"Saved batch {file_count} with {len(doc_buffer)} docs to {batch_path}")
                        doc_buffer = []

                except json.JSONDecodeError:
                    print(f"Skipping malformed line in {file}")
                except Exception as e:
                    print(f"Error processing line in {file}: {str(e)}")

if doc_buffer:
    file_count += 1
    batch_path = os.path.join(PARSED_DIR, f"parsed_docs_{file_count:04d}.json")
    with open(batch_path, 'w', encoding='utf-8') as out_f:
        json.dump(doc_buffer, out_f)
    print(f"Saved final batch {file_count} with {len(doc_buffer)} docs to {batch_path}")

print(f"Total documents parsed: {doc_count}")
print(f"Total batch files written: {file_count}")

Saved batch 1 with 10000 docs to parsed_docs_batches/parsed_docs_0001.json
Saved batch 2 with 10000 docs to parsed_docs_batches/parsed_docs_0002.json
Saved batch 3 with 10000 docs to parsed_docs_batches/parsed_docs_0003.json
Saved batch 4 with 10000 docs to parsed_docs_batches/parsed_docs_0004.json
Saved batch 5 with 10000 docs to parsed_docs_batches/parsed_docs_0005.json
Saved batch 6 with 10000 docs to parsed_docs_batches/parsed_docs_0006.json
Saved batch 7 with 10000 docs to parsed_docs_batches/parsed_docs_0007.json
Saved batch 8 with 10000 docs to parsed_docs_batches/parsed_docs_0008.json
Saved batch 9 with 10000 docs to parsed_docs_batches/parsed_docs_0009.json
Saved batch 10 with 10000 docs to parsed_docs_batches/parsed_docs_0010.json
Saved batch 11 with 10000 docs to parsed_docs_batches/parsed_docs_0011.json
Saved batch 12 with 10000 docs to parsed_docs_batches/parsed_docs_0012.json
Saved final batch 13 with 2191 docs to parsed_docs_batches/parsed_docs_0013.json
Total documents 

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import os

# --- Config ---
PARSED_DIR = "parsed_docs_batches"
CHUNKED_DIR = "chunked_docs_batches"
os.makedirs(CHUNKED_DIR, exist_ok=True)

MAX_CHUNKS_PER_FILE = 250_000
MAX_FILE_SIZE_MB = 150

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=200,  
    separators=["\n\n", "\n", " ", ""],
    length_function=len
)

batch_files = sorted([
    f for f in os.listdir(PARSED_DIR)
    if f.startswith("parsed_docs_") and f.endswith(".json")
])

chunk_file_idx = 1
chunk_count = 0
file_bytes = 0
total_chunks = 0
out_f = open(os.path.join(CHUNKED_DIR, f"chunked_docs_{chunk_file_idx:04d}.jsonl"), "w", encoding="utf-8")

for batch_file in batch_files:
    input_path = os.path.join(PARSED_DIR, batch_file)

    with open(input_path, 'r', encoding='utf-8') as f_in:
        parsed_docs = json.load(f_in)

        for doc_idx, doc in enumerate(parsed_docs):
            try:
                chunks = text_splitter.split_text(doc["text"])

                for chunk_index, chunk in enumerate(chunks):
                    chunk_obj = {
                        "source": doc["source"],
                        "chunk": chunk,
                        "doc_index": doc_idx,
                        "chunk_index": chunk_index
                    }
                    json_str = json.dumps(chunk_obj)
                    byte_len = len(json_str.encode("utf-8"))

                    if chunk_count >= MAX_CHUNKS_PER_FILE or file_bytes + byte_len > MAX_FILE_SIZE_MB * 1024 * 1024:
                        out_f.close()
                        chunk_file_idx += 1
                        out_f = open(os.path.join(CHUNKED_DIR, f"chunked_docs_{chunk_file_idx:04d}.jsonl"), "w", encoding="utf-8")
                        chunk_count = 0
                        file_bytes = 0

                    out_f.write(json_str + "\n")
                    chunk_count += 1
                    file_bytes += byte_len
                    total_chunks += 1

            except Exception as e:
                print(f"Error processing doc {doc_idx} in {batch_file}: {e}")

# Final flush
if not out_f.closed:
    out_f.close()

print(f"Chunking complete. Total chunks: {total_chunks}")
print(f"Output files written to '{CHUNKED_DIR}'")

Chunking complete. Total chunks: 16884492
Output files written to 'chunked_docs_batches'


In [5]:
# !pip install --upgrade snowflake-snowpark-python
from snowflake.snowpark import Session
import getpass

password = getpass.getpass("Enter Snowflake password: ")

connection_parameters = {
    "account": "SFEDU02-PDB57018",
    "user": "CAT",
    "password": password,
    "role": "TRAINING_ROLE",
    "warehouse": "DAMG7374",
    "database": "LAWS_CONTRACTS",
    "schema": "TEXT"
}

session = Session.builder.configs(connection_parameters).create()
print("Connected to Snowflake!")

Connected to Snowflake!


In [6]:
import json
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

CHUNKED_DIR = "chunked_docs_batches"
BATCH_FILES = sorted(f for f in os.listdir(CHUNKED_DIR) if f.endswith(".jsonl"))

def upload_to_snowflake(file):
    path = os.path.join(CHUNKED_DIR, file)
    try:
        with open(path, "r", encoding="utf-8") as f:
            records = [json.loads(line) for line in f if line.strip()]
        if not records:
            return f"Skipped empty: {file}"

        df = session.create_dataframe(records, schema=["SOURCE", "CHUNK", "DOC_INDEX", "CHUNK_INDEX"])
        df.write.mode("append").save_as_table("DOCS_CHUNKS_TABLE")
        return f"Uploaded {len(records)} chunks from {file}"
    except Exception as e:
        return f"Error uploading {file}: {e}"

# --- Run uploads in parallel ---
MAX_WORKERS = 8
print(f"Starting concurrent upload with {MAX_WORKERS} workers...")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {executor.submit(upload_to_snowflake, file): file for file in BATCH_FILES}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Uploading to Snowflake"):
        print(future.result())

Starting concurrent upload with 8 workers...


Uploading to Snowflake:   1%|          | 1/96 [01:55<3:03:14, 115.73s/it]

Uploaded 180733 chunks from chunked_docs_0006.jsonl


Uploading to Snowflake:   2%|▏         | 2/96 [01:59<1:18:21, 50.02s/it] 

Uploaded 179965 chunks from chunked_docs_0003.jsonl


...

Uploaded 92523 chunks from chunked_docs_0096.jsonl


Uploading to Snowflake:  99%|█████████▉| 95/96 [19:37<00:05,  5.59s/it]

Uploaded 170785 chunks from chunked_docs_0095.jsonl


Uploading to Snowflake: 100%|██████████| 96/96 [19:38<00:00, 12.27s/it]

Uploaded 172612 chunks from chunked_docs_0094.jsonl
